# Creating Cleaned Movies dataset for Data Exploration

In [1]:
import pandas as pd
import datetime
import numpy as np
pd.set_option('display.max_columns', 23)

In [2]:
movies=pd.read_csv('../Data/movies.csv')

In [3]:
movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
print(datetime.datetime.now())

for i,row in movies.iterrows():
    for x in row['genres'].split("|"):
        movies.at[i,x]=1
        
movies=movies.fillna(0)
print(datetime.datetime.now())

2018-08-04 17:57:41.736036
2018-08-04 17:57:44.558484


#Time consuming method

print(datetime.datetime.now())

for i,row in movies.iterrows():
    for x in row['genres'].split("|"):
        movies[x]=0
        
for i,row in movies.iterrows():
    for x in row['genres'].split("|"):
        movies.at[i,x]=1
        
print(datetime.datetime.now())

#Takes 10 seconds more for 15 M records

In [5]:
pd.set_option('display.max_columns', 23)
movies.head(5)

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
col_list=movies.columns.values[3:]
col_list

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

In [7]:
movies['Genre_Count'] = movies[col_list].sum(axis=1)

In [8]:
movies.to_csv('../Output/CleanedMovies.csv')

# Create Ratings Sample

In [9]:
ratings=pd.read_csv('../Data/ratings.csv')

In [10]:
#Find the lowest count of movies for a specific rating to get a downsampling size
sampleSize=ratings.groupby("rating").count().reset_index()['userId'].min()
sampleSize

239125

In [11]:
#Saving unique ratings into a numpy array
value=ratings['rating'].unique()
value

array([3.5, 4. , 3. , 4.5, 5. , 2. , 1. , 2.5, 0.5, 1.5])

In [12]:
RatingSampledf=pd.DataFrame()

In [13]:
for ratingValue in value:
    filtereddf = ratings.loc[lambda x: x['rating'] == ratingValue]
    sampledf=filtereddf.sample(sampleSize)
    RatingSampledf=pd.concat([RatingSampledf, sampledf],axis=0)
print("Sample created")

Sample created


In [14]:
len(RatingSampledf)

2391250

In [15]:
RatingSampledf.to_csv("../Output/SampleRatings.csv")

# Create a WordCloud for genres of top 100 movies with maximum ratings

In [16]:
#Select top 100 values using 'nlargest'
top100ratings=ratings.groupby('movieId').count().nlargest(100,'rating')

In [17]:
movie_rating=pd.merge(top100ratings,movies, on='movieId', how='inner')

In [18]:
#Convert dataframe to list
Genres=movie_rating['genres'].values.tolist()

In [19]:
#Join list values
Genres=",".join(Genres).join(("",""))

In [20]:
#Replace '|' with ',' and then replace ',' with ',\n' to save the entire dataset into an excel
GenresString=Genres.replace('|',',')
GenresString=GenresString.replace(',',',\n')

In [21]:
f = open('../Output/GenreString.csv','w')
f.write(GenresString) #Give your csv text here.
## Python will convert \n to os.linesep
f.close()